In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

import matplotlib.pyplot as plt

from datetime import datetime

In [2]:
# Load and examine the dataset
df = pd.read_csv('/Users/rotation/Desktop/Dataclass/capstone-project/archive_boston/crime.csv', encoding='latin-1')
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [3]:
# Examine available columns
df.columns.tolist()

['INCIDENT_NUMBER',
 'OFFENSE_CODE',
 'OFFENSE_CODE_GROUP',
 'OFFENSE_DESCRIPTION',
 'DISTRICT',
 'REPORTING_AREA',
 'SHOOTING',
 'OCCURRED_ON_DATE',
 'YEAR',
 'MONTH',
 'DAY_OF_WEEK',
 'HOUR',
 'UCR_PART',
 'STREET',
 'Lat',
 'Long',
 'Location']

In [4]:
df.apply(pd.Series.nunique)


INCIDENT_NUMBER        282517
OFFENSE_CODE              222
OFFENSE_CODE_GROUP         67
OFFENSE_DESCRIPTION       244
DISTRICT                   12
REPORTING_AREA            879
SHOOTING                    1
OCCURRED_ON_DATE       233229
YEAR                        4
MONTH                      12
DAY_OF_WEEK                 7
HOUR                       24
UCR_PART                    4
STREET                   4657
Lat                     18178
Long                    18178
Location                18194
dtype: int64

In [5]:
df.drop_duplicates(subset="INCIDENT_NUMBER", inplace=True)
df.drop_duplicates(subset="OFFENSE_CODE", inplace=True)
df.drop_duplicates(subset="OFFENSE_CODE", inplace=True)
df.drop_duplicates(subset="OFFENSE_DESCRIPTION", inplace=True)

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
OFFENSE_CODE,202.0,2052.787129,1082.227728,111.000000,1102.750000,2008.500000,3007.750000,3831.000000
YEAR,202.0,2017.782178,0.655690,2015.000000,2018.000000,2018.000000,2018.000000,2018.000000
MONTH,202.0,8.113861,1.642685,1.000000,8.000000,8.000000,9.000000,12.000000
HOUR,202.0,13.455446,6.226288,0.000000,11.000000,14.000000,18.000000,23.000000
Lat,199.0,41.890192,4.332555,-1.000000,42.302025,42.331521,42.350142,42.378773
Long,199.0,-70.378957,7.008211,-71.169499,-71.093959,-71.078664,-71.062154,-1.000000


In [7]:
# Get transpose of DataFrame
transposedDf = df.T
print(transposedDf)

                                          0                            1       \
INCIDENT_NUMBER                       I182070945                   I182070943   
OFFENSE_CODE                                 619                         1402   
OFFENSE_CODE_GROUP                       Larceny                    Vandalism   
OFFENSE_DESCRIPTION           LARCENY ALL OTHERS                    VANDALISM   
DISTRICT                                     D14                          C11   
REPORTING_AREA                               808                          347   
SHOOTING                                     NaN                          NaN   
OCCURRED_ON_DATE             2018-09-02 13:00:00          2018-08-21 00:00:00   
YEAR                                        2018                         2018   
MONTH                                          9                            8   
DAY_OF_WEEK                               Sunday                      Tuesday   
HOUR                        

In [8]:
#df.groupby(['INCIDENT_NUMBER','DISTRICT']).count()


In [9]:
# FREQUENCY
print(df.OFFENSE_CODE_GROUP.value_counts())


Drug Violation                     25
Other                              21
Motor Vehicle Accident Response    11
Larceny                             9
Robbery                             8
                                   ..
Warrant Arrests                     1
Service                             1
Auto Theft Recovery                 1
Landlord/Tenant Disputes            1
Burglary - No Property Taken        1
Name: OFFENSE_CODE_GROUP, Length: 66, dtype: int64


In [10]:
# create a dict of lists
data = {'OFFENSE_CODE_GROUP': ['Larceny',
 'Vandalism',
 'Towed',
 'Investigate Property',
 'Motor Vehicle Accident Response',
 'Auto Theft',
 'Verbal Disputes',
 'Robbery',
 'Fire Related Reports',
 'Other',
 'Property Lost',
 'Medical Assistance',
 'Assembly or Gathering Violations',
 'Larceny From Motor Vehicle',
 'Residential Burglary',
 'Simple Assault',
 'Violations',
 'Harassment',
 'Ballistics',
 'Property Found',
 'Police Service Incidents',
 'Drug Violation',
 'Missing Person Reported',
 'Investigate Person',
 'Fraud',
 'Aggravated Assault',
 'License Plate Related Incidents',
 'Other Burglary',
 'Warrant Arrests',
 'Disorderly Conduct',
 'Harbor Related Incidents',
 'Counterfeiting',
 'Liquor Violation',
 'Firearm Discovery',
 'Landlord/Tenant Disputes',
 'Auto Theft Recovery',
 'Service',
 'Operating Under the Influence',
 'Confidence Games',
 'Restraining Order Violations',
 'Firearm Violations',
 'Missing Person Located',
 'License Violation',
 'Commercial Burglary',
 'Search Warrants',
 'Recovered Stolen Property',
 'Offenses Against Child / Family',
 'Prostitution',
 'Bomb Hoax',
 'Evading Fare',
 'Property Related Damage',
 'Prisoner Related Incidents',
 'Homicide',
 'Embezzlement',
 'Explosives',
 'Arson',
 'Criminal Harassment',
 'HOME INVASION',
 'Phone Call Complaints',
 'Aircraft',
 'Biological Threat',
 'Manslaughter',
 'Gambling',
 'HUMAN TRAFFICKING',
 'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE',
 'Burglary - No Property Taken'],
       'OFFENSE_CODE':[619,
 1402,
 3410,
 3114,
 3820,
 724,
 3301,
 301,
 3108,
 2647,
 3201,
 3006,
 3305,
 614,
 522,
 802,
 2907,
 2629,
 2662,
 3207,
 3109,
 1843,
 3501,
 3115,
 1102,
 423,
 3205,
 561,
 3125,
 2405,
 3116,
 1001,
 2646,
 3119,
 3112,
 735,
 3402,
 2101,
 1106,
 2007,
 1503,
 3502,
 3111,
 540,
 3130,
 1304,
 2005,
 1605,
 2648,
 2632,
 3106,
 2619,
 111,
 1201,
 3123,
 900,
 2670,
 2010,
 2628,
 3122,
 2672,
 121,
 1901,
 1610,
 1620,
 527]}
        
        
# Creating a dataframe from the dict
df = pd.DataFrame(data)

df.head()

,OFFENSE_CODE_GROUP,OFFENSE_CODE
0,Larceny,619
1,Vandalism,1402
2,Towed,3410
3,Investigate Property,3114
4,Motor Vehicle Accident Response,3820


In [11]:
from scipy.stats import mode

def unique_nan(s):
    return s.nunique(dropna=False)
def count_nulls(s):
    return s.size - s.count()

agg_func_custom_count = {
    'OFFENSE_CODE_GROUP': ['count', 'nunique', 'size', unique_nan, count_nulls, set]
}
df.groupby(['DAY_OF_WEEK']).agg(agg_func_custom_count)

KeyError: 'DAY_OF_WEEK'

In [12]:
df.drop_duplicates(subset="OFFENSE_CODE", inplace=True)
df.OFFENSE_CODE.tolist()

[619,
 1402,
 3410,
 3114,
 3820,
 724,
 3301,
 301,
 3108,
 2647,
 3201,
 3006,
 3305,
 614,
 522,
 802,
 2907,
 2629,
 2662,
 3207,
 3109,
 1843,
 3501,
 3115,
 1102,
 423,
 3205,
 561,
 3125,
 2405,
 3116,
 1001,
 2646,
 3119,
 3112,
 735,
 3402,
 2101,
 1106,
 2007,
 1503,
 3502,
 3111,
 540,
 3130,
 1304,
 2005,
 1605,
 2648,
 2632,
 3106,
 2619,
 111,
 1201,
 3123,
 900,
 2670,
 2010,
 2628,
 3122,
 2672,
 121,
 1901,
 1610,
 1620,
 527]

In [13]:
df.drop_duplicates(subset="OFFENSE_CODE_GROUP", inplace=True)
df.OFFENSE_CODE_GROUP.tolist()

['Larceny',
 'Vandalism',
 'Towed',
 'Investigate Property',
 'Motor Vehicle Accident Response',
 'Auto Theft',
 'Verbal Disputes',
 'Robbery',
 'Fire Related Reports',
 'Other',
 'Property Lost',
 'Medical Assistance',
 'Assembly or Gathering Violations',
 'Larceny From Motor Vehicle',
 'Residential Burglary',
 'Simple Assault',
 'Violations',
 'Harassment',
 'Ballistics',
 'Property Found',
 'Police Service Incidents',
 'Drug Violation',
 'Missing Person Reported',
 'Investigate Person',
 'Fraud',
 'Aggravated Assault',
 'License Plate Related Incidents',
 'Other Burglary',
 'Warrant Arrests',
 'Disorderly Conduct',
 'Harbor Related Incidents',
 'Counterfeiting',
 'Liquor Violation',
 'Firearm Discovery',
 'Landlord/Tenant Disputes',
 'Auto Theft Recovery',
 'Service',
 'Operating Under the Influence',
 'Confidence Games',
 'Restraining Order Violations',
 'Firearm Violations',
 'Missing Person Located',
 'License Violation',
 'Commercial Burglary',
 'Search Warrants',
 'Recovered St